# Guide to Pynattas (old version)

NOTE: assuming dependencies already in place. A guide for dependencies will be created on a later date. For now, make sure to have the latest compatible versions of:
- pytorch and pytorch lightning, torchmetrics and torchvision, cuda
- tensorflow and tensorboard
- tqdm
- datetime
- matplotlib and nump

Solve any further incompatibilities as they are raised.

## Run the code with current configuration

NOTE: There are possible combinations of blocks that will crash the run. A saner block selection logic will be added in the future.

In [ ]:
import pynattas as pnas

In [ ]:
%run main.py

## GUIDE TO CONFIG.INI

NOTE: most sanity checks are still missing. Be careful please.

- In the [Mode] section, select the desired mode for pynattas. If both are true, nas will be done first, then ht will be done on the winning architecture of nas. If only ht is true, then tuning will be done on the architecture written in architecture_code under the [NAS] section.
- max_layers in [NAS] section refers to the maximum size of the chromosomes (aka, maximum number of convolutions-type layers).
- In the [GA] section, change the parameters for NAS search.
- In the [Computation] section, be careful about the num_workers value. Currently set to 1 for safety when testing parallelization. In computation, also change the accellerator to either "cpu" or "gpu".


## GUIDE TO PARALLELIZATION (currently broken)

NOTE: Mostly untested.
- In main.py, change line 54 ("nas_result = pnas.optimizers.ga.ga_optimizer(") to:
    - nas_result = pnas.optimizers.ga_concurrent.ga_optimizer(" for nas parallelization using a ThreadPoolExecutor
    - nas_result = pnas.optimizers.ga_concurrent_pp.ga_optimizer(" for nas parallelization using a ProcessPoolExecutor

Currently, it looks like there are issues with ProcessPoolExecutor. Regardless, be careful when selecting num_workers in the config.ini when testing these.

## TO ADD NEW BLOCKS

- in pynattas.blocks add the new block to the related .py file depending on type.
- in config.ini add the new block as a section with all related parameters. Use "default_" for default values that are used during NAS, and "min_" and "max_" for range values to be used during tuning. Also, update the commented vocabulary at the start for clarity purposes.
- in pynattas.configuration.vocabulary, update relative vocabularies.
- in pynattas.classes.generic_network, add the layer construction to the list making sure that the current_layers, current_height, and current_width is calculated correctly.
    

## TO WORK ON NEW DATASET

- Add dataset to the "data" folder
- Add compatible datamodule to the "datasets" folder
- in pynattas.functions.fitness, import the desired datamodule. Also, around line 59, update the datahandling to be compatible with your datamodule. For the ClassificationHead, Make sure the data is resized to 256x256 sized HxW images, that the number of channels passed to the model is correct.
- change config.ini in the [Dataset] section. "data_path" should point to the dataset, "csv_path" is for a .csv for labels if required. If it's not required, imput None

## GUIDE TO LOGS

Logs are stored in the "logs" folder and in the "lightning_logs" folder. In "logs", you will find:
- logs about NAS iterations and results in the GA_logs subfolder
- logs about HT iterations and results in either the GWO_logs or PSO_logs subfolder depending on the selected HT algorithm
- logs about the final run comprised of the saved checkpoint .ckpt file in the "tb_logs" subfolder, together with all the confusion matrixes generated.

## INFERENCE (Coming soon...)